<a href="https://colab.research.google.com/github/xxx0708/CNN/blob/main/cifar10class6_modify1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
import os
import shutil#进行复制和移动文件或者文件夹
import random
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets
import torchvision.transforms as transforms
#from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import sampler

to_float = torch.float#浮点型数据类型
to_long = torch.long

if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print(device)

cpu


# 定义网络模型

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.net = nn.Sequential(
            
    nn.Conv2d(3,16,3,padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2,stride=2),
            
    nn.Conv2d(16,32,3,padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2,stride=2),
            
    nn.Conv2d(32,32,3,padding=1),      
    nn.ReLU(),
    nn.MaxPool2d(2,stride=2),
            
    nn.Flatten(),
    nn.Linear(32*4*4,32*2*2),
    nn.Linear(32*2*2,10)
)  
    def forward(self, x):
        return self.net(x)

# 打印准确率函数

In [ ]:
def check_accuracy_part34(loader, model):
  #if loader.dataset.train:
  #  print('Checking accuracy on validation set')
  #else:
  #  print('Checking accuracy on test set')   
  num_correct = 0
  num_samples = 0
  model.eval()  # set model to evaluation mode
  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device, dtype=to_float)  # move to device, e.g. GPU
      y = y.to(device=device, dtype=to_long)
      scores = model(x)
      _, preds = scores.max(1)
      num_correct += (preds == y).sum()
      num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
  return acc

# 加载数据集

In [ ]:
#train_data = datasets.ImageFolder('C:/Users/Administrator/Desktop/RecCifar10to6-master/train')
#test_data = datasets.ImageFolder('C:/Users/Administrator/Desktop/RecCifar10to6-master/test')
# 训练集
train_data = datasets.CIFAR10(root='./CIFAR10', train=True, download=True, transform=transform)
 #测试集
test_data = datasets.CIFAR10(root='./CIFAR10', train=False, download=True, transform=transform)
NUM_TRAIN = 49000

# 随机提取6类训练5次，最终结果取平均

In [ ]:
main_dir = os.getcwd()
train_dir = main_dir + "/extract_train"
test_dir = main_dir + "/extract_test"

epochs = 10
total = []

for i in range(1):
    #建立存放数据集的文件夹
    if os.path.exists(train_dir) and os.path.exists(test_dir):
        shutil.rmtree(train_dir)
        shutil.rmtree(test_dir)
        os.mkdir(train_dir)
        os.mkdir(test_dir)
    else:
        os.mkdir(train_dir)
        os.mkdir(test_dir)
        
    #生成6个随机数，用于提取数据集中的6类
    class_6 = torch.randperm(10)[:6]
    print("Extract classes: ", class_6, "\n")

    #提取6类并保存至新的文件夹
    for i in range(50000):
        train_label = train_data.imgs[i][1]

        if train_label in class_6:
            train_label_dir = train_dir + "/" + str(train_label)
            if os.path.exists(train_label_dir):
                pass
            else:
                os.mkdir(train_label_dir)
            image_path = train_data.imgs[i][0]
            image = Image.open(image_path)
            save_path = train_label_dir + "/" + str(i) + ".jpg"
            image.save(save_path)

    for i in range(10000):
        test_label = test_data.imgs[i][1]

        if test_label in class_6:
            test_label_dir = test_dir + "/" + str(test_label)
            if os.path.exists(test_label_dir):
                pass
            else:
                os.mkdir(test_label_dir)
            image_path = test_data.imgs[i][0]
            image = Image.open(image_path)
            save_path = test_label_dir + "/" + str(i) + ".jpg"
            image.save(save_path)
            
    #定义预处理操作
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
        ])

    #数据集预处理
    train_data_ = datasets.ImageFolder(train_dir, transform=transform)
    test_data_ = datasets.ImageFolder(test_dir, transform=transform)

    #数据集采样、打包
    n_train = len(train_data_)  #30000

    split = 22375
    indices = list(range(n_train))
    random.shuffle(indices)

    train_loader = torch.utils.data.DataLoader(train_data_, batch_size=18, 
                                              sampler=sampler.SubsetRandomSampler(indices[:split]))
    valid_loader = torch.utils.data.DataLoader(train_data_, batch_size=18, 
                                            sampler=sampler.SubsetRandomSampler(indices[split:]))
    test_loader = torch.utils.data.DataLoader(test_data_, batch_size=18, shuffle=True)
    
    #定义网络与参数更新方法
    model = Net()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    
    #训练网络
    num_prints = epochs * len(train_loader) // 100 + 1
    acc_history = torch.zeros(num_prints, dtype=to_float)
    iter_history = torch.zeros(num_prints, dtype=to_long)
    model = model.to(device=device)# move the model parameters to CPU/GPU
    for epoch in range(epochs):
        for t, (x, y) in enumerate(train_loader):
            model.train()
            # Move the data to the proper device (GPU or CPU)
            x = x.to(device=device, dtype=to_float)
            y = y.to(device=device, dtype=to_long)

            # Forward pass: compute scores and loss
            scores = model(x)
            loss = F.cross_entropy(scores, y)

            optimizer.zero_grad()

            #Backwards pass: compute the gradient
            loss.backward()

            #Update the parameters
            optimizer.step()

            tt = t + epoch * len(train_loader)

            if tt % 100 == 0 or (epoch == epochs - 1 and t == len(train_loader) - 1):
                print("Epoch: %d, Iteration: %d, loss = %.4f" %(epoch, tt, loss.item()))
                acc = check_accuracy_part34(valid_loader, model)
                acc_history[tt // 100] = acc
                iter_history[tt // 100] = tt
                print()
    test_acc = check_accuracy_part34(test_loader, model)
    print()
    total.append(test_acc)
    
final_acc = sum(total) / len(total) * 100
print("Final accuracy: %.2f" %final_acc)

Extract classes:  tensor([5, 6, 7, 8, 2, 0]) 



D:\Anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 0, Iteration: 0, loss = 2.3194
Got 1244 / 7625 correct (16.31)

Epoch: 0, Iteration: 100, loss = 2.0496
Got 1397 / 7625 correct (18.32)

Epoch: 0, Iteration: 200, loss = 1.8167
Got 1515 / 7625 correct (19.87)

Epoch: 0, Iteration: 300, loss = 1.6627
Got 2104 / 7625 correct (27.59)

Epoch: 0, Iteration: 400, loss = 1.6915
Got 2659 / 7625 correct (34.87)

Epoch: 0, Iteration: 500, loss = 1.5427
Got 2716 / 7625 correct (35.62)

Epoch: 0, Iteration: 600, loss = 1.4021
Got 2807 / 7625 correct (36.81)

Epoch: 0, Iteration: 700, loss = 1.2953
Got 2741 / 7625 correct (35.95)

Epoch: 0, Iteration: 800, loss = 1.6789
Got 3406 / 7625 correct (44.67)

Epoch: 0, Iteration: 900, loss = 1.4939
Got 3263 / 7625 correct (42.79)

Epoch: 0, Iteration: 1000, loss = 1.3810
Got 3415 / 7625 correct (44.79)

Epoch: 0, Iteration: 1100, loss = 1.2979
Got 3460 / 7625 correct (45.38)

Epoch: 0, Iteration: 1200, loss = 1.3779
Got 3626 / 7625 correct (47.55)

Epoch: 1, Iteration: 1300, loss = 1.1382
Got 3546 

Epoch: 8, Iteration: 11100, loss = 0.7382
Got 5583 / 7625 correct (73.22)

Epoch: 9, Iteration: 11200, loss = 0.5924
Got 5584 / 7625 correct (73.23)

Epoch: 9, Iteration: 11300, loss = 0.7405
Got 5508 / 7625 correct (72.24)

Epoch: 9, Iteration: 11400, loss = 0.7026
Got 5468 / 7625 correct (71.71)

Epoch: 9, Iteration: 11500, loss = 0.7667
Got 5595 / 7625 correct (73.38)

Epoch: 9, Iteration: 11600, loss = 0.6314
Got 5485 / 7625 correct (71.93)

Epoch: 9, Iteration: 11700, loss = 0.4455
Got 5606 / 7625 correct (73.52)

Epoch: 9, Iteration: 11800, loss = 0.4998
Got 5348 / 7625 correct (70.14)

Epoch: 9, Iteration: 11900, loss = 0.6480
Got 5498 / 7625 correct (72.10)

Epoch: 9, Iteration: 12000, loss = 0.7391
Got 5443 / 7625 correct (71.38)

Epoch: 9, Iteration: 12100, loss = 0.8446
Got 5261 / 7625 correct (69.00)

Epoch: 9, Iteration: 12200, loss = 0.6646
Got 5585 / 7625 correct (73.25)

Epoch: 9, Iteration: 12300, loss = 0.8363
Got 5653 / 7625 correct (74.14)

Epoch: 9, Iteration: 1240

# 随机提取6类进行单次训练

## 数据集加载

In [ ]:
#定义预处理操作
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
    ])

#数据集预处理
train_data_ = datasets.ImageFolder(train_dir, transform=transform)
test_data_ = datasets.ImageFolder(test_dir, transform=transform)

#数据集采样、打包
n_train = len(train_data_)  #30000

split = 22375
indices = list(range(n_train))
random.shuffle(indices)

train_loader = torch.utils.data.DataLoader(train_data_, batch_size=18, 
                                          sampler=sampler.SubsetRandomSampler(indices[:split]))
valid_loader = torch.utils.data.DataLoader(train_data_, batch_size=18, 
                                        sampler=sampler.SubsetRandomSampler(indices[split:]))
test_loader = torch.utils.data.DataLoader(test_data_, batch_size=18, shuffle=True)

## 训练网络

In [ ]:
#定义网络与参数更新方法
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01)

epochs = 10
num_prints = epochs * len(train_loader) // 100 + 1
acc_history = torch.zeros(num_prints, dtype=to_float)
iter_history = torch.zeros(num_prints, dtype=to_long)
model = model.to(device=device)# move the model parameters to CPU/GPU
for epoch in range(epochs):
    for t, (x, y) in enumerate(train_loader):
        model.train()
        # Move the data to the proper device (GPU or CPU)
        x = x.to(device=device, dtype=to_float)
        y = y.to(device=device, dtype=to_long).clamp(max=5)
        
        # Forward pass: compute scores and loss
        scores = model(x)
        loss = F.cross_entropy(scores, y)
        
        optimizer.zero_grad()
        
        #Backwards pass: compute the gradient
        loss.backward()
        
        #Update the parameters
        optimizer.step()
        
        tt = t + epoch * len(train_loader)
        
        if tt % 100 == 0 or (epoch == epochs - 1 and t == len(train_loader) - 1):
            print("Epoch: %d, Iteration: %d, loss = %.4f" %(epoch, tt, loss.item()))
            acc = check_accuracy_part34(valid_loader, model)
            acc_history[tt // 100] = acc
            iter_history[tt // 100] = tt
            print()

Epoch: 0, Iteration: 0, loss = 2.3103
Got 1258 / 7625 correct (16.50)

Epoch: 0, Iteration: 100, loss = 1.8604
Got 1247 / 7625 correct (16.35)

Epoch: 0, Iteration: 200, loss = 1.7329
Got 2108 / 7625 correct (27.65)

Epoch: 0, Iteration: 300, loss = 1.6297
Got 2503 / 7625 correct (32.83)

Epoch: 0, Iteration: 400, loss = 1.8208
Got 2414 / 7625 correct (31.66)

Epoch: 0, Iteration: 500, loss = 1.6169
Got 2959 / 7625 correct (38.81)

Epoch: 0, Iteration: 600, loss = 1.5495
Got 2824 / 7625 correct (37.04)

Epoch: 0, Iteration: 700, loss = 1.3642
Got 2825 / 7625 correct (37.05)

Epoch: 0, Iteration: 800, loss = 1.3751
Got 3208 / 7625 correct (42.07)

Epoch: 0, Iteration: 900, loss = 1.5438
Got 3529 / 7625 correct (46.28)

Epoch: 0, Iteration: 1000, loss = 1.0877
Got 3312 / 7625 correct (43.44)

Epoch: 0, Iteration: 1100, loss = 1.2534
Got 3560 / 7625 correct (46.69)

Epoch: 0, Iteration: 1200, loss = 1.3544
Got 3704 / 7625 correct (48.58)

Epoch: 1, Iteration: 1300, loss = 1.2800
Got 3735 

In [ ]:
import matplotlib.pyplot as plt
plt.title('Val accuracies')
plt.plot(iter_history, acc_history, '-o')
plt.xlabel('iterations')
plt.ylabel('accuracy')
plt.show()